In [3]:
import psycopg2

#### Dlaczego polecam bardziej bibliotekę psycopg2 niż SQLAlchemy:

- Idealny dla programistów/analityków, którzy preferują pisanie surowych zapytań SQL i potrzebują wysokiej wydajności.

Należy pamiętać, że są pewne ogranicznenia:

- Ograniczony zakres: Zaprojektowany wyłącznie dla PostgreSQL, co oznacza, że nie można go używać z innymi systemami baz danych.


**SQLAlchemy:**
- Biblioteka ORM (Object Relational Mapping), która zapewnia wyższy poziom abstrakcji nad interakcjami z bazą danych.
- Odpowiednia dla tych, którzy chcą pracować z bazami danych za pomocą obiektów Pythona zamiast zapytań SQL.

In [2]:
# !pip install psycopg2-binary

In [5]:
# POŁĄCZENIE Z BAZĄ
conn = psycopg2.connect(host='localhost', database="postgres", user="postgres", password="admin", port=5432)

In [6]:
cur = conn.cursor()

In [7]:
cur.execute("""create table IF NOT EXISTS players (
  player_id serial primary key,
  first_name text not null,
  last_name text not null,
  height numeric not null,
  weight numeric not null,
  salary numeric not null
);
""")

In [8]:
conn.commit()

In [ ]:
cur.execute(""" insert into players (first_name,last_name,height,weight, salary) values
('Marian', 'Nowak', 1.90, 80, 10000),
('Jan', 'Kowalski', 1.80, 75, 150000),
('Chuck', 'Norris', 1.70, 58, 300000);
""")
conn.commit()

In [10]:
cur.execute("""SELECT * FROM players;""")
print(cur.fetchall())

[(1, 'Marian', 'Nowak', Decimal('1.90'), Decimal('80'), Decimal('10000')), (2, 'Jan', 'Kowalski', Decimal('1.80'), Decimal('75'), Decimal('150000')), (3, 'Chuck', 'Norris', Decimal('1.70'), Decimal('58'), Decimal('300000'))]


In [11]:
cur.execute("""SELECT * FROM players;""")
for row in cur.fetchall():
    print(row)

(1, 'Marian', 'Nowak', Decimal('1.90'), Decimal('80'), Decimal('10000'))
(2, 'Jan', 'Kowalski', Decimal('1.80'), Decimal('75'), Decimal('150000'))
(3, 'Chuck', 'Norris', Decimal('1.70'), Decimal('58'), Decimal('300000'))


In [12]:
# NA KONIEC PRACY Z BAZĄ DANYCH
cur.close()
conn.close()